In [ ]:
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

# Set seeds for reproducibility
np.random.seed(20)
random.seed(20)
tf.random.set_seed(20)

# Read and preprocess training data
train_data_path = 'train_data.csv'  # Update with the correct path
train_df = pd.read_csv(train_data_path, delimiter=';', header=None, skiprows=1, names=['ORB', 'SSIM', 'VGG16', 'GRADED'])

# Normalize features
scaler = MinMaxScaler()
X_train = scaler.fit_transform(train_df[['ORB', 'SSIM', 'VGG16']])
y_train = train_df['GRADED'].values

# Define and compile the model
model = Sequential([
    Dense(10, input_dim=3, activation='relu'),  # Input layer with 3 inputs, hidden layer with 24 neurons
    Dense(10, activation='relu'),  # Second hidden layer
    Dense(1, activation='linear')  # Output layer with a single output (grade)
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# Use early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with fewer epochs and a larger validation split
model.fit(X_train, y_train, epochs=200, validation_split=0.3, callbacks=[early_stopping], verbose=1)

# Load and preprocess the test data
test_data_path = 'tekk.csv'  # Update this path with the correct test data location
test_df = pd.read_csv(test_data_path, delimiter=';', header=None, skiprows=1, names=['ORB', 'SSIM', 'VGG16', 'GRADED'])

# Use the same scaler to normalize test features
X_test = scaler.transform(test_df[['ORB', 'SSIM', 'VGG16']])
y_test = test_df['GRADED'].values

# Evaluate the model on the test data
loss, mae = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {loss}, Test MAE: {mae}")

# Predict on the test data
predicted_grades = model.predict(X_test, verbose=1)
test_df['PREDICTED_GRADE'] = predicted_grades

# Save predictions to a new CSV
test_df.to_csv('test_data_with_predictions3.csv', index=False)

print(f"Predictions saved to 'test_data_with_predictions.csv'")


: 